In [4]:
## Imports 

import numpy as np
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
import string
from keras.optimizers import Adam, SGD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression
import math
import cv2
from os.path import join
from os import getcwd


##

## Task 1: training two models

## Model 1: Multilayer perceptron 

## import data set 
path = join( getcwd(), 'data', 'training-dataset.npz' ) 

with np.load(path) as data: 
    
    img = data['x']
    lbl = data['y']
    
X_train, X_test, y_train, y_test = tts(img, lbl, test_size=0.2)
X_train, X_val, y_train, y_val = tts(X_train, y_train, test_size=0.25)

print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

unique_labels = len(set(lbl))
labels = set(lbl)
print("Number of labels:", unique_labels)
print("Output classes:", labels)

onehot = LabelBinarizer()
y_train_hot = onehot.fit_transform(y_train)
y_val_hot = onehot.transform(y_val)
y_test_hot = onehot.transform(y_test)

inp_shape = X_train.shape[1]

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(inp_shape,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(unique_labels, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
run_model = model.fit(X_train, y_train_hot, batch_size=256, epochs=1, verbose=1,
                   validation_data=(X_val, y_val_hot))

[test_loss, test_acc] = model.evaluate(X_test, y_test_hot)

print("Results model with 4 layers: Loss = {}, accuracy = {}".format(test_loss, test_acc))

##

## Model 2: Neural Network

x, x_test, y, y_test = tts(img, lbl, test_size=0.15, train_size=0.85) 
x_train, x_val, y_train, y_val = tts(x, y, test_size=0.17647059, train_size=0.82352941) 

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

LB = LabelBinarizer()
Y_train = LB.fit_transform(y_train)
Y_val   = LB.transform(y_val)
Y_test  = LB.transform(y_test)

model2 = Sequential()
model2.add(Dense(512, input_dim=784, activation='relu'))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(26, activation='softmax')) #last layer

optimizer = Adam(lr=0.001)
model2.compile(optimizer=optimizer, loss = "categorical_crossentropy", metrics=['accuracy']) 
model2.fit(x_train, Y_train, batch_size=32, epochs=1, verbose=1)

y_pred = model2.predict(x_val)
print('MAE:', mean_absolute_error(Y_val, y_pred).round(3))
print('MSE:', mean_squared_error(Y_val, y_pred).round(3))

baseline = LogisticRegression()
baseline.fit(x_val, y_val)

print('Accuracy Validation:', accuracy_score(y_val, baseline.predict(x_val)).round(3))

[test_loss, test_acc] = model2.evaluate(x_test, Y_test)
print("Results: Loss = {}, accuracy = {}".format(test_loss, test_acc))

Training data shape: (74880, 784) (74880,)
Testing data shape: (24960, 784) (24960,)
Number of labels: 26
Output classes: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}
780/780 [==============================] - 5s 6ms/step - loss: 0.7969 - accuracy: 0.7741
Results model with 4 layers: Loss = 0.7968903183937073, accuracy = 0.7740785479545593
2730/2730 [==============================] - 44s 16ms/step - loss: 0.6532 - accuracy: 0.8043
MAE: 0.032
MSE: 0.032


C:\Users\CJBS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy Validation: 0.768
585/585 [==============================] - 3s 6ms/step - loss: 0.4577 - accuracy: 0.8592
Results: Loss = 0.45771318674087524, accuracy = 0.8591880202293396


In [ ]:
## Task two: Running on multiple letters within image

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    #return resized
    # define padding 
    if resized.shape[0] < height: 
        pad = (height - resized.shape[0]) /2
        tpad = math.ceil(pad) # round up
        bpad = int(pad) # round down
        lpad, rpad = 0,0 
        
    if resized.shape[1] < width:
        pad = (width - resized.shape[1])/2
        lpad = math.ceil(pad) # round up
        rpad = int(pad)
        tpad, bpad = 0, 0 
        
    image = cv2.copyMakeBorder(resized, tpad, bpad, lpad, rpad, # top, bottom, left, right,
        cv2.BORDER_CONSTANT)
   
    return image

def find_contour(dic): #dictionary with rectangle contour & index as key
    
    #Step 2: sort dictionary with entry with longest width
        ###NOTE: Assumption - Overlapping letters are of longest width 
    longest_width = sorted(dic.values(), key=lambda x:x[2])[-1] #longest one last 

    #Step 3: Create new contours 
    new_contours = {}
    counter = 0 

    for ind in range( len(dic) ): 

        #separate letters 
        if dic[ind] == longest_width: 

            #setup
            x, y, w, h = dic[ind] 

            #first letter 
            r1 = x, y, int(w/2), h 
            new_contours[counter] = r1
            counter +=1 

            #second letter 
            r2 = int(x + w/2), y, int(w/2), h
            new_contours[counter] = r2     
            counter +=1 
            

        else: 
            new_contours[counter]= dic[ind]
            counter +=1   
                   
    return new_contours # contours_found (for testing > shows change)

def unwanted(dic): #List of array of contours// assumes contour more than 4 
        
    # list of array by how many too long 
    no_excess = len(dic) - 4
    
    #list showing array and size of respective array
    l = []
    for contour in dic.values():
        
        #setup
        x, y, w, h = contour

        l.append( (contour, (w*h)) )
        

    #create new list with excess array
    excess =  sorted(l, key=lambda x:x[-1])[:no_excess]
    re_list = []
    
    for item in excess:
        
        re_list.append(item[0])

    #Step 3: Create new contours 
    new_contours = {}
    counter = 0 

    for val in dic.values():
        
        if val in re_list: 
            
            continue 
            
        else: 
            new_contours[counter] = val
            counter +=1
            
    return new_contours

def find_4contours(contours_found): # dic with index keys 
    
    if len( contours_found ) < 4:
            
            new_contours = find_contour(contours_found) # here recursion would be good 
            
    else:
            
            new_contours = unwanted(contours_found) #removes excess array based on size
            
    return new_contours

def check(contours_found):
    
    if len(contours_found) == 4: 
        
        return contours_found
    
    else: 
        return check(find_4contours(contours_found))
    

#load dataset 
path = join( getcwd(), 'data', 'test-dataset.npy' ) 
captchas = np.load(path).astype('uint8') # test_ll = test_labelless

#iterate through case by case: 
final_pred = []
css = []

for exp, captcha in enumerate (captchas): 
    
    #------format captcha image-----#
    
    ##add extra padding around the image
    pad = cv2.copyMakeBorder(captcha, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    
    ## blur 
    blur = cv2.medianBlur(pad , 3)
    
    ##thresholding 
    _, thresh = cv2.threshold(blur,127,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) #127 middle between white/black
    
    
    #------contour extraction-----#
   
    ##detect contours of all images 
    contours, hierachy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    #print (contours, f"hierachy: {hierachy}")
    #------contour filtration-----#
    
    ##contours smaller than 30 pixels excluded
            ##NOTE - replacing pixel with black may lead to imporvement
    
    contours_letters = []
    
    for contour in contours:
        
        css.append(contour.size)

        if contour.size > 30: # check whats the best size 
            
            contours_letters.append(contour)
            
            
    #------letter extraction-----#  
    
        #Prep: Build dic with contours and index as key
        
    contours_extracted = [] #stores contours found: rectangle
                        #e.g. [(103, 9, 20, 17), ...]
    
    for i in range( len(contours_letters) ):
        
        contours_extracted.append(cv2.boundingRect(contours_letters[i]))
        
        
    #------sort contours-----# 
     ## sorted in accordance with x position
    sort_x = sorted(contours_extracted, key=lambda x: x[0])
    
    ##index key position
    contours_found = {}
    
    for ind, contour in enumerate( sort_x ):
        
        contours_found[ind] = contour
        
    #--contour rectangle adjust--#
    
        #Main part: Finding relevant 4 contours if 
        #we do not have more or less than 4 contours
    
    new_contours = check( contours_found ) #this works 
        
    #-------prediction-------
    
        #take dictionary with index as key in form {0: (103, 9, 20, 17), ...}
    
    captcha = ""
    
    for key in new_contours.keys():
        
        x, y, w, h = new_contours[key]
        
        # Extract the letter from the original image with a 2-pixel margin around the edge
        #letter_image = pad[y - 2:y + h + 2, x - 2:x + w + 2]
        letter_image = pad[y:y + h, x:x + w]
        
        #resize
        #l_I2828 = image_resize(letter_image, 28, 28)
        #l_I2828 = imutils.resize(letter_image, width=28, height=28)
        l_I2828 = cv2.resize(letter_image, (28, 28), interpolation=cv2.INTER_AREA)
                #interpolation=cv2.INTER_AREA ==> for decimating/ resizing image default INTER_LINEAR 
                # https://www.geeksforgeeks.org/image-resizing-using-opencv-python/
                #https://docs.opencv.org/3.4.0/da/d54/group__imgproc__transform.html#gga5bb5a1fea74ea38e1a5445ca803ff121acf959dca2480cc694ca016b81b442ceb
                
        
        #Normalisation/ reshape
        K = l_I2828/255.0  # Normalisation
        K = K.reshape(-1,784)
        
        #prediction
        pred = model.predict(K)
        
        #append list 
        letter = onehot.inverse_transform(pred)[0]
        
        #correct format
        if letter < 10:
            captcha += "0" + str( letter )
        else:
            captcha += str( letter ) 
    
    print(captcha)
    final_pred.append(int(captcha))
##

## Extra: Predict function to use on trained models

def predict_letter(testcase, model, xtest, ytest): # input is a number between 0 and 24960
    model_name = model
    alphabet = list(string.ascii_lowercase) # list of alphabet letters for clarity
    prediction = alphabet[model_name.predict_classes(xtest[[testcase], :])[0]] # use the model to predict a one hot label, convert to letter
    true_label = alphabet[ytest[testcase] -1] # true label from y_test one hot, converted to a letter
    
    print("Letter prediction {}".format(prediction))
    plt.imshow(X_test[testcase].reshape((28, 28)), cmap='gray') # plot the letter for clarity
    plt.title("True label: {}".format(true_label))
    plt.show()
    print("Actual letter: {}".format(true_label))
    if true_label == prediction:
        print("The model got it correct")
    else:
        print("The model got it wrong")
    
    return
    
#predict_letter(863, model, X_test, y_test)

##

17071723
17070524
06010604
08080117
07142402
17062302
07042307
23041223
23230207
11080623
13010221
17060605
24171705
08241112
07232307
23082416
06221708
06081316
23050123
04011523
11062323
23242404
05102324
07060503
01231101
08081706
24231317
23130417
06041723
07151713
08072305
02230404
24080707
08211124
04231705
02240602
08050604
04230404
05211702
13240704
24250407
01232323
08081625
23101105
24172324
17170506
03010723
24230806
17171223
04240406
23072317
05102406
04041706
07070714
05230706
08231702
07052423
23230508
24082423
02012324
23242311
07040624
08012407
08051305
24082317
23230105
01042305
17172416
23050807
07242306
12080412
23172217
07082423
07230823
17010424
24231717
17232310
17240605
23230807
17231704
17072606
23170823
01231723
21010617
23120725
26082408
07170611
07062305
17240623
23110623
08080613
23081708
01052407
23072307
05172323
24061122
17072417
04170101
13170723
02080608
17242424
07240222
23172406
19042308
06060723
06231706
23010812
08221417
07241724
05242317
23041323
1

01111715
05080413
16072406
07110824
02240405
12230523
08010817
23130117
17012417
13221701
23170717
07112423
02230825
05040511
17170604
17080508
02172317
17072311
08241724
17040522
07080608
07042308
06241108
22240207
23081720
02072305
19060717
10070819
23080612
08070124
04062417
25081317
23060824
17070517
05040402
17171723
23070804
07011324
17051724
17072324
04080623
04071723
06041318
17020706
04052508
23232421
17131123
04170101
23172302
05080808
06232306
24240207
07230806
13240808
07232323
06131708
01171301
05240606
07071306
07232305
23232307
07250406
23231325
17172324
07250404
05241101
07072317
23010724
01012308
01221308
23172313
17010824
25230613
23232301
23010707
11242407
06051306
08012107
08012306
23230623
22172501
24080407
07130713
08230823
17041702
17072304
06230417
05242304
10040623
08232223
05230825
04040106
24172425
02081316
08171005
11242320
11232324
24172312
23062313
08072301
04171706
17010707
23130717
06230423
17040817
17062304
05230802
07170823
06240604
08141705
01210404
1

24230605
17230808
24232307
12012023
23200623
04011907
04081706
17232306
08070423
17242408
01241701
01240623
02170423
24250206
17172207
23242407
08082325
01232524
23020124
24072323
13251117
23060806
17081707
17242326
07130805
23121608
11231207
04041723
08072326
11262324
25010723
24222317
04232501
05252422
24112604
23172302
21220424
07081623
23242324
07230104
07072324
21070624
17040623
08170708
22171707
24080623
25172317
24232423
03111224
24112506
25072305
08021703
24051424
24130606
08072317
24082317
06262317
06242323
23060807
17070126
23050415
17051701
23162224
04242323
05132407
04231723
04171308
23080806
23232407
13052324
01172308
24012323
24230617
05070824
02071704
07062304
08170405
08230723
17081206
10242307
07232308
07251124
06022307
07062007
06070117
08170607
25011924
15242306
11230723
08232323
06170407
23171701
07012404
04262324
07112604
07060124
04022406
07012301
11230616
17172522
10110607
22010602
13111708
08172307
17072217
06170816
24232424
17080606
23021708
04041207
08242308
0